# Выбор локации для скважины

Мне предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов.

Задача:
- Постройть модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль.
- Проанализировать возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

<div id='1'>Ключевые константы</div>

In [2]:
RANDOM_STATE = 12345
MAX_INVEST = 10_000_000_000
INCOME_PER_1000 = 450_000
MAX_LOSS_RISK = .025
N_TEST = 500
N_DEPT = 200

## Загрузка и подготовка данных

In [3]:
# загрузим данные по регионам
try:
    reg_0, reg_1, reg_2 = (pd.read_csv(f'/datasets/geo_data_{i}.csv') for i in range(3))
except FileNotFoundError:
    reg_0, reg_1, reg_2 = (pd.read_csv(f'geo_data_{i}.csv') for i in range(3))

In [4]:
reg_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


Пропусков нет - дополнительной обработки отсутствующих значений не требуется

In [5]:
reg_2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


Признаки не нормализованы:
- среднее арифметическое не равно 0,
- среднее квадратическое отклонение не  равно 1.  

Следует провести нормализацию признаков.

## Обучение и проверка модели

In [11]:
# создадим конвейр по нормализации данных и вычислению линейной регрессии
model = Pipeline([
    ('scaler', StandardScaler()),
    ('linear_regression', LinearRegression())])
model_data = {}
for i, data in enumerate((reg_0, reg_1, reg_2)):
    # получаем целевые значения и признаки для каждого региона
    target = data['product']
    features = data[['f0', 'f1', 'f2']]
    # разделяем данные на обучающие и валидационные
    X_train, X_valid, y_train, y_valid = train_test_split(features, target, test_size=.25,
                                                    random_state=RANDOM_STATE)
    # обучим модель и сделаем предсказание на валидационных признаках
    y_pred = model.fit(X_train, y_train).predict(X_valid)
    # сохраним предсказания и истинные значения в словаре
    model_data[f'reg_{i}'] = pd.DataFrame({'pred': y_pred, 'true': y_valid})
    print(f'Средний предсказанный запас из региона {i}: {y_pred.mean():.2f}')
    print(f'Средняя квадратическая ошибка: {mean_squared_error(y_valid, y_pred, squared=False):.2f}')
    print(f'Коэффициенты: a0 = {model.named_steps.linear_regression.coef_[0]:.2f},',
          f'a1 = {model.named_steps.linear_regression.coef_[1]:.2f},',
          f'a2 = {model.named_steps.linear_regression.coef_[2]:.2f}')
    print()

Средний предсказанный запас из региона 0: 92.59
Средняя квадратическая ошибка: 37.58
Коэффициенты: a0 = 3.13, a1 = -7.11, a2 = 21.43

Средний предсказанный запас из региона 1: 68.73
Средняя квадратическая ошибка: 0.89
Коэффициенты: a0 = -1.30, a1 = -0.11, a2 = 45.89

Средний предсказанный запас из региона 2: 94.97
Средняя квадратическая ошибка: 40.03
Коэффициенты: a0 = 0.05, a1 = -0.07, a2 = 19.82



**Анализ моделей:**
- Наиболее `точно` модели удалось описать зависимости месторождение `reg_1` (`RMSE=0.89`)
- Наибольшая `положительная связь` отмечена с признаком `f2` (у `reg_1` выражена наиболее сильно - `45.89`)
- `Наибольший` средний предсказанный `запас` у месторождения `reg_2` (`94.97`), но у этой модели также и `очень большая` среднеквадратическая `ошибка` (`RMSE=40.03`)

## Подготовка к расчёту прибыли

### Достаточный объём сырья для безубыточной разработки.

Формула среднего объёма сырья для безубыточной добычи (1000 барелей):  
<center><font size=6>$Инвестиции \over Стоимость_{1000 барелей} x Количество_{скважин}$</font></center>

In [47]:
# минимальный безубыточный уровень добычи со скважины
min_drill_dept = MAX_INVEST / (INCOME_PER_1000 * N_DEPT)
min_drill_dept

111.11111111111111

### Выводы по этапу подготовки расчёта прибыли.

- Для `возврата инвестиций` произведение `объёма добычи` сырья со всех скважин `на цену сырья` должен равняться `общему размеру` инвестиций.
- При заданных объеме инвестиций, доходу от продажи сырья и количестве скважин `средний объём` с одной скважины `должен` быть `111 тысяч барелей`.
- `Все` месторождения имеют `более низкие средние объемы сырья` (некоторые скважины могут оказаться вообще пустыми).
- Для отбора лучших скважин количество скважин разведки (500) будет превышать количество скважин добычи (200).

## Расчёт прибыли и рисков 

In [48]:
def drills_true_income(df, n_test=N_TEST, n_dept=N_DEPT, credit=MAX_INVEST,
                       price=INCOME_PER_1000, random_state=None):
    """
    Возвращает прибыль от добычи нефти из выбранных скважин.
    Параметры:
    df - датафрейм с предсказанными и реальными объёмами добычи
    n_test - количество разведочных скважин
    n_dept - количество скважин добычи с наибольшими объёмами
    credit - понесенные затраты
    price - цена нефти
    random_state - фиксация псевдослучайности
    """
    drills_choosen = (df
                      # случайная выборка заданного количества разведочных данных
                      .sample(n=n_test, replace=True, random_state=state)
                      # сортировка скважин с наибольшим объёмом добычи
                      .sort_values('pred', ascending=False)
                      # отбор заданного количества скважин добычи
                      .iloc[:n_dept])
    return drills_choosen['true'].sum() * price -credit 

In [49]:
state = np.random.RandomState(RANDOM_STATE)

for i in range(3):
    # собираем в датафрейм доходы от 200 лучших из 500 случайных скважин
    profits = pd.Series([drills_true_income(model_data[f'reg_{i}'],
                                      random_state=state) for _ in range(1000)])
    profit_mean = profits.mean()
    risk = (profits < 0).mean()
    lower_limit = profits.quantile(.025)
    upper_limit = profits.quantile(.975)
    print(f'Характеристики месторождения reg_{i}:', '-' * 30, sep='\n')
    print(f'Средняя прибыль = {profit_mean:_.0f}')
    print(f'Границы 95% доверительного интервала:')
    print(f'- нижняя (0.025 квантиль) = {lower_limit:_.0f} руб')
    print(f'- верхняя (0.975 квантиль) = {upper_limit:_.0f} руб')
    print(f'Риск убытков = {risk:.2%}', '*' * 30, sep='\n')

Характеристики месторождения reg_0:
------------------------------
Средняя прибыль = 396_164_985
Границы 95% доверительного интервала:
- нижняя (0.025 квантиль) = -111_215_546 руб
- верхняя (0.975 квантиль) = 909_766_942 руб
Риск убытков = 6.90%
******************************
Характеристики месторождения reg_1:
------------------------------
Средняя прибыль = 461_155_817
Границы 95% доверительного интервала:
- нижняя (0.025 квантиль) = 78_050_811 руб
- верхняя (0.975 квантиль) = 862_952_060 руб
Риск убытков = 0.70%
******************************
Характеристики месторождения reg_2:
------------------------------
Средняя прибыль = 392_950_475
Границы 95% доверительного интервала:
- нижняя (0.025 квантиль) = -112_227_625 руб
- верхняя (0.975 квантиль) = 934_562_915 руб
Риск убытков = 6.50%
******************************


**Выводы:**
- Рекомендованное месторождение `reg_1` по причинам:
 - наиболее высокой средней `прибыли` от месторождения `461_155_817`
 - положительных значений нижней и верхней границ `доверительного интервала`
 - низкого `риска` невозврата инвестиций (`0.7%` при допустимом риске 2.5%)
 - модель по месторождению показывает `наилучшую точность`.